In [1]:
# Do the necessary imports

import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print('setup complete')

setup complete


In [2]:
df = pd.read_csv('pathi_test.csv')

In [3]:
df.head()

,id,user,android_sensor_gyroscope_mean,android_sensor_accelerometer_std,android_sensor_gyroscope_std,android_sensor_gyroscope_uncalibrated_mean,android_sensor_accelerometer_max,android_sensor_linear_acceleration_mean,speed_mean,android_sensor_rotation_vector_mean,android_sensor_rotation_vector_max,android_sensor_accelerometer_min,android_sensor_magnetic_field_uncalibrated_min,sound_min,target
0,16170,Luca,0.001651,0.014626,0.000737,0.016221,9.849411,0.020978,0.000000,0.050413,0.056351,9.758895,51.199707,NaN,Still
1,15871,Luca,0.036326,1.775944,0.020290,0.039023,17.146631,0.879220,16.539349,0.999981,0.999999,7.707437,82.409890,89.065143,Car
2,16811,Luca,0.001525,0.011199,0.000713,0.016302,9.849262,0.034110,0.000000,0.610456,0.610456,9.804817,55.501802,NaN,Still
3,15831,Luca,0.036440,0.862553,0.010553,0.050759,12.304298,1.488361,17.739895,0.998112,0.998112,7.659674,95.664309,87.470377,Car
4,876,andrea,0.183202,0.504117,0.098819,0.265652,10.891645,1.658308,9.000000,0.322242,0.378193,8.965621,156.795909,89.770732,Car


In [4]:
print(len(df))
df.nunique()

5893


id                                                5893
user                                                13
android_sensor_gyroscope_mean                     5018
android_sensor_accelerometer_std                  5861
android_sensor_gyroscope_std                      4926
android_sensor_gyroscope_uncalibrated_mean        5000
android_sensor_accelerometer_max                  5838
android_sensor_linear_acceleration_mean           4972
speed_mean                                         646
android_sensor_rotation_vector_mean               5002
android_sensor_rotation_vector_max                4991
android_sensor_accelerometer_min                  5833
android_sensor_magnetic_field_uncalibrated_min    4707
sound_min                                         2306
target                                               5
dtype: int64

### We need to separate users from train and test. (we need atleast 1180 rows of data thats not in train)

#### Drop few users data such that the train data is 80% of original data

In [5]:
df['user'].unique().tolist()

['Luca',
 'andrea',
 'Federica',
 'michelangelo',
 'Damiano',
 'Claudio',
 'Vincenzo',
 'Serena',
 'Pierpaolo',
 'IvanHeibi',
 'AndreaCarpineti',
 'Elena',
 'Riccardo']

In [6]:
#TRAIN DATA

#drop these users from train data so that the model will never see them in test

luca_index = df[df['user'] == 'Luca'].index
Ivan_index = df[df['user'] == 'IvanHeibi'].index
Pier_index = df[df['user'] == 'Pierpaolo'].index
Riccardo_index = df[df['user'] == 'Riccardo'].index

# Delete these row indexes from dataFrame
train_df = df.drop(luca_index)
train_df = train_df.drop(Ivan_index)
train_df = train_df.drop(Pier_index)
train_df = train_df.drop(Riccardo_index)

print(len(train_df))
train_df.head()

4454


,id,user,android_sensor_gyroscope_mean,android_sensor_accelerometer_std,android_sensor_gyroscope_std,android_sensor_gyroscope_uncalibrated_mean,android_sensor_accelerometer_max,android_sensor_linear_acceleration_mean,speed_mean,android_sensor_rotation_vector_mean,android_sensor_rotation_vector_max,android_sensor_accelerometer_min,android_sensor_magnetic_field_uncalibrated_min,sound_min,target
4,876,andrea,0.183202,0.504117,0.098819,0.265652,10.891645,1.658308,9.00,0.322242,0.378193,8.965621,156.795909,89.770732,Car
5,13480,Federica,NaN,0.103857,NaN,NaN,9.520044,NaN,0.00,NaN,NaN,9.149029,NaN,57.361127,Car
6,18786,michelangelo,0.004100,0.013489,0.001801,0.042991,9.816197,0.027441,NaN,0.201198,0.201318,9.758751,170.521377,65.158369,Train
8,343,andrea,0.041554,0.255053,0.030074,0.065754,10.027802,0.278997,4.75,0.780795,0.786845,8.996134,134.611517,89.808441,Bus
9,4650,andrea,0.037451,0.267791,0.021391,0.030491,10.068528,0.255172,NaN,0.968082,0.968806,9.014786,140.174223,89.815738,Train


In [7]:
train_df['user'].unique()

array(['andrea', 'Federica', 'michelangelo', 'Damiano', 'Claudio',
       'Vincenzo', 'Serena', 'AndreaCarpineti', 'Elena'], dtype=object)

In [8]:
#TEST DATA

andrea_index = df[df['user'] == 'andrea'].index
federica_index = df[df['user'] == 'Federica'].index
michel_index = df[df['user'] == 'michelangelo'].index
damiano_index = df[df['user'] == 'Damiano'].index
claudio_index = df[df['user'] == 'Claudio'].index
vincenzo_index = df[df['user'] == 'Vincenzo'].index
serena_index = df[df['user'] == 'Serena'].index
andreacarpi_index = df[df['user'] == 'AndreaCarpineti'].index
elena_index = df[df['user'] == 'Elena'].index


# Delete these row indexes from dataFrame
test_df = df.drop(andrea_index)
test_df = test_df.drop(federica_index)
test_df = test_df.drop(michel_index)
test_df = test_df.drop(damiano_index)
test_df = test_df.drop(claudio_index)
test_df = test_df.drop(vincenzo_index)
test_df = test_df.drop(serena_index)
test_df = test_df.drop(andreacarpi_index)
test_df = test_df.drop(elena_index)


print(len(test_df))
test_df.head()

1439


,id,user,android_sensor_gyroscope_mean,android_sensor_accelerometer_std,android_sensor_gyroscope_std,android_sensor_gyroscope_uncalibrated_mean,android_sensor_accelerometer_max,android_sensor_linear_acceleration_mean,speed_mean,android_sensor_rotation_vector_mean,android_sensor_rotation_vector_max,android_sensor_accelerometer_min,android_sensor_magnetic_field_uncalibrated_min,sound_min,target
0,16170,Luca,0.001651,0.014626,0.000737,0.016221,9.849411,0.020978,0.000000,0.050413,0.056351,9.758895,51.199707,NaN,Still
1,15871,Luca,0.036326,1.775944,0.020290,0.039023,17.146631,0.879220,16.539349,0.999981,0.999999,7.707437,82.409890,89.065143,Car
2,16811,Luca,0.001525,0.011199,0.000713,0.016302,9.849262,0.034110,0.000000,0.610456,0.610456,9.804817,55.501802,NaN,Still
3,15831,Luca,0.036440,0.862553,0.010553,0.050759,12.304298,1.488361,17.739895,0.998112,0.998112,7.659674,95.664309,87.470377,Car
7,15268,Luca,0.140902,1.044684,0.119667,0.142145,14.202603,1.003520,7.797239,0.929187,0.941615,6.431646,58.646911,NaN,Bus


In [9]:
test_df['user'].unique()

array(['Luca', 'Pierpaolo', 'IvanHeibi', 'Riccardo'], dtype=object)

In [10]:
train_df['target'].unique()

array(['Car', 'Train', 'Bus', 'Still', 'Walking'], dtype=object)

In [11]:
#split the features and target in train data

X_train = train_df.drop(['target', 'id','user'], axis = 1)
y_train = train_df.target
#X_train.head()

y_train = y_train.replace({'Still':0, 'Walking':1, 'Car':2, 'Bus':3, 'Train':4})

In [12]:
#split the features and target in test data

X_test = test_df.drop(['target', 'id','user'], axis = 1)
y_test = test_df.target


y_test = y_test.replace({'Still':0, 'Walking':1, 'Car':2, 'Bus':3, 'Train':4})

In [13]:
#numerical columns in the train data

num_vars  = list(X_train.columns)
num_vars

['android_sensor_gyroscope_mean',
 'android_sensor_accelerometer_std',
 'android_sensor_gyroscope_std',
 'android_sensor_gyroscope_uncalibrated_mean',
 'android_sensor_accelerometer_max',
 'android_sensor_linear_acceleration_mean',
 'speed_mean',
 'android_sensor_rotation_vector_mean',
 'android_sensor_rotation_vector_max',
 'android_sensor_accelerometer_min',
 'android_sensor_magnetic_field_uncalibrated_min',
 'sound_min']

In [14]:
# Pipeline for Tree models

num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)),
  ('Normalizer',RobustScaler())
  
])


tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
], remainder='passthrough') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(remainder='passthrough',
                  transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('Normalizer',
                                                  RobustScaler())]),
                                 ['android_sensor_gyroscope_mean',
                                  'android_sensor_accelerometer_std',
                                  'android_sensor_gyroscope_std',
                                  'android_sensor_gyroscope_uncalibrated_mean',
                                  'android_sensor_accelerometer_max',
                                  'android_sensor_linear_acceleration_mean',
                                  'speed_mean',
                                  'android_sensor_rotation_vector_mean',
                                  'android_sensor_rotation_vector_max',
                                  'android_sensor_accelerometer_min',
                                  'android_sensor_magnetic_field_uncalibrated_min',
                                  'sound_min'])])

In [15]:
#import the tress models

from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
#from xgboost               import XGBClassifier
#from lightgbm              import LGBMClassifier
#from catboost              import CatBoostClassifier

In [16]:
#create a dict with the models that we want to train on

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier()
  #"XGBoost":XGBClassifier(),
  #"LightGBM":LGBMClassifier(),
  #"CatBoost":CatBoostClassifier()
}

In [17]:
#pass the pipeline to everymodel

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

In [ ]:
#y_train

In [18]:
#fit the data to the model

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    X_train, y_train,
    test_size=0.002,
    stratify = y_train,   #is something wrong here?
    random_state=37
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(X_test)  
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

,Model,Accuracy,Bal Acc.,Time
1,Random Forest,47.046560,45.131604,1.738240
2,Skl HistGBM,46.560111,45.072230,14.790275
3,Skl GBM,44.961779,43.472070,14.765625
4,Extra Trees,42.946491,40.872433,0.690944
5,Decision Tree,41.209173,42.105247,0.088416
6,AdaBoost,41.139680,41.526299,0.882121


# WHERE IS THE DATA LEAAAAAAK????

# send help

In [19]:
from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(y_test, pred))

[[130   2 164   5   7]
 [ 25 394  43   1   1]
 [  1   2 131   0  11]
 [  4  32 202   0  94]
 [  2   2 171   0  15]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3e7089f3-11a5-48ad-89c4-39a166311a14' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>